In [1]:
import pandas as pd

In [2]:
df = pd.DataFrame({"cat": ["food", "food", "snack", "snack", "snack"],
                   "type": ["A", "A", "B", "C", "B"],
                   "price": [100, 100, 300, 200, 500],
                   "star": [3.1, 3.5, 4.3, 4, 3.9]})
df

,cat,type,price,star
0,food,A,100,3.1
1,food,A,100,3.5
2,snack,B,300,4.3
3,snack,C,200,4.0
4,snack,B,500,3.9


.groupby() 메서드는 메서드 내부에 데이터프레임의 특정 기준 변수명을 입력하여 해당 변수를 기준으로 다른 변수를 요약 연산을 실시할 수 있도록 지원하는 메서드. 기준 변수의 경우 그 개수가 하나일 경우 단순 문자열로 입력하면 되나, 두 개 이상인 경우 변수명을 리스트로 감싼다. 그리고 요약 연산의 대상이 되는 변수는 메서드 뒤에 대괄호를 사용하여 명시한다. 마지막으로 연산 규칙 또는 알고리즘은 대괄호 뒤에 적절한 메서드(.mean(), .agg() 등)를 사용하여 지정한다.

In [3]:
df.groupby("cat")["star"].mean()

cat
food     3.300000
snack    4.066667
Name: star, dtype: float64

.groupby() 메서드 내부에 명시한 변수명은 각 변수의 원소 고유값이 인덱스로 지정되며 단일 변수를 요약하는 경우 그 결과가 Pandas Series객체로 반환된다.

In [4]:
df.groupby("cat")[["price", "star"]].mean()

,price,star
cat,,
food,100.000000,3.300000
snack,333.333333,4.066667


In [5]:
df.groupby("cat")["price", "star"].mean() # 권장x

C:\Users\Encaion\AppData\Local\Temp\ipykernel_3712\1262868425.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df.groupby("cat")["price", "star"].mean() # 권장x


,price,star
cat,,
food,100.000000,3.300000
snack,333.333333,4.066667


두 개 이상의 변수를 대상으로 연산하는 경우 .groupby() 메서드 뒤의 대괄호 내부에 대상 변수명을 리스트로 감싸야 한다. 리스트로 감싸지 않고 쉼표로만 구분하는 경우는 tuple로 인식되어 처리가 되기때문에 "FutureWarning"이 발생함.

In [6]:
df.groupby(["cat", "type"])["price"].mean()

cat    type
food   A       100.0
snack  B       400.0
       C       200.0
Name: price, dtype: float64

In [8]:
# df.groupby("cat", "type")["price"].mean() # 에러!

.groupby() 메서드 내부에 2개 이상의 기준 변수를 지정할 경우 리스트로 감싸주지 않으면 에러가 발생한다.

In [10]:
df.groupby(["cat", "type"]).mean()

price  star
cat   type             
food  A     100.0   3.3
snack B     400.0   4.1
      C     200.0   4.0

상기와 같이 .groupby() 메서드 뒤에 연산 대상 변수를 지정하지 않은 경우 메서드 내부에서 언급한 변수를 제외한 나머지 모든 변수가 연산에 사용된다.

In [12]:
df.groupby(["cat", "type"])["price"].count()

cat    type
food   A       2
snack  B       2
       C       1
Name: price, dtype: int64

.count() 메서드는 .groupby() 메서드 내부에 2개 이상의 기준 변수를 지정한 경우 인덱스는 .drop_duplicates() 메서드로 중복제거한 것과 같은 모양이며 각 인덱스별 등장 횟수를 확인할 수 있음. 즉, 고유한 조합이 데이터프레임에 몇 번 등장하는지 확인 가능.

대상 변수 뒤에 사용하는 메서드의 경우 대부분 Pandas Series 객체의 메서드를 지원하며, 지원하지 않는 특수한 연산을 수행하고자 하는 경우는 .agg() 메서드와 lambda 함수 또는 사용자 정의 함수를 사용해야 함. 그리고 .agg() 메서드는 여러 연산을 한 번에 계산할 수 있도록 지원한다.

In [14]:
df.groupby("cat")["price"].agg("mean")

cat
food     100.000000
snack    333.333333
Name: price, dtype: float64

In [15]:
df.groupby("cat")["price"].agg(["mean", "max"])

,mean,max
cat,,
food,100.000000,100
snack,333.333333,500


In [16]:
def round_mean(x): # 기준변수 기준으로 쪼개어진 대상변수(Series)가 입력으로 처리됨.
    return round(x.mean(), 2)

df.groupby("cat")["price"].agg(round_mean)

cat
food     100.00
snack    333.33
Name: price, dtype: float64

In [20]:
# df.groupby("cat")["price"].agg([mean, max]) # 에러!!!

In [18]:
# df.groupby("cat")["price"].agg("round_mean") # 에러!!!

In [21]:
df.groupby("cat")["price"].agg(["mean", "max", round_mean]) # 굳굳.

,mean,max,round_mean
cat,,,
food,100.000000,100,100.00
snack,333.333333,500,333.33


.groupby() 메서드는 내부에 지정되는 기준변수를 기준으로 메서드 뒤에 기술하는 대상 변수를 쪼개어 처리함.

In [22]:
df.groupby("cat")["price"].agg(lambda x: round(x.mean(), 2))

cat
food     100.00
snack    333.33
Name: price, dtype: float64

lambda 함수는 별도로 정의하지 않고 임시로 쓰는 일회성 함수. 한 줄 정도로 해결되는 간단한 일회성의 사용자 정의 함수가 필요할 때 주로 사용.

In [24]:
df.groupby("cat")["price"].mean().reset_index()

,cat,price
0,food,100.000000
1,snack,333.333333


In [26]:
df.groupby(["cat", "type"])[["price", "star"]].mean().reset_index()

,cat,type,price,star
0,food,A,100.0,3.3
1,snack,B,400.0,4.1
2,snack,C,200.0,4.0


Series 또는 인덱스 기반 연산이 불편하다면 .reset_index() 메서드를 활용하여 인덱스를 초기화 하면서 Series를 DataFrame으로 바꾸거나 인덱스를 별도의 변수로 배치시킬 수 있다.